### Week 2 – Manipulation et formatage avancé de fichiers Excel

#### Objectifs
- Insérer des formules Excel depuis Python
- Contrôler les plages de cellules (sélection de lignes, colonnes, zones)
- Appliquer des formats : gras, couleurs, formats monétaires
- Générer des tableaux Excel professionnels automatiquement

#### Chargement d’un fichier Excel avec openpyxl

In [43]:
from openpyxl import load_workbook
import pandas as pd
# Charger le fichier Excel
wb = load_workbook("Week 1 V2.xlsx")
ws = wb.active
# Afficher le nom de la feuille active
print("Feuille active :", ws.title)
# Renommer la feuille
ws.title = "Données employés"
# Vérifier le changement
print("Nouveau nom de feuille active :", ws.title)
df.to_excel("Week 1.xlsx", sheet_name="Données employés", index=False)
wb.save("Week 1 V2.xlsx")

Feuille active : Sheet1
Nouveau nom de feuille active : Données employés


#### Sélection de cellules, lignes et colonnes
En `openpyxl`, on peut sélectionner :

- Une cellule : `ws["B2"]`
- Une ligne : `ws[2]` ou `ws.iter_rows(min_row=2, max_row=2)`
- Une colonne : `ws["B"]`
- Une plage : `ws["A2:C5"]` (plage rectangulaire)

In [33]:
# Une cellule
cellule = ws["B2"]
print("B2 :", cellule.value)
# Une plage de cellules
plage = ws["A2:C3"]
for row in plage:
    print([cell.value for cell in row])
# Une colonne entière
for cell in ws["C"]:
    # Sans les cellules vides
    if cell.value is not None:
        print(cell.value)

B2 : 23
['Alice', 23, 35000]
['Charlie', 37, 60000]
Salaire_net
35000
60000


In [34]:
# Vérifie si la cellule D1 contient déjà l'en-tête "Salaire_brut"
if ws["D1"].value != "Salaire_brut":
    ws["D1"] = "Salaire_brut"
    # Applique la formule dans toute les lignes de la colonne D
    for y in range(2, ws.max_y + 1):
        ws[f"D{y}"] = f"=C{y}*1.25"
else:
    print(f"Colonne '{ws['D1'].value}' déjà présente, pas de modification.")

Colonne 'Salaire_brut' déjà présente, pas de modification.


Remarque :
- y est une variable temporaire qui va prendre les valeurs 2, 3, 4, … jusqu’à ws.max_y.
- Ce nom n’a pas d’importance fonctionnelle, tant que tu l’utilises de manière cohérente dans le bloc de la boucle.

#### Gérer les feuilles Excel avec `openpyxl`

- Créer une nouvelle feuille : `wb.create_sheet("Nom")`
- Dupliquer une feuille : `wb.copy_worksheet(feuille)`
- Supprimer une feuille : `wb.remove(feuille)`

In [63]:
from openpyxl import load_workbook
import pandas as pd

# Charger le fichier
wb = load_workbook("Week 1 v2.xlsx")

# Créer une nouvelle feuille vide
new_sheet = wb.create_sheet("Résumé")
print(wb.sheetnames)

# Dupliquer une feuille existante
copie = wb.copy_worksheet(wb["Résumé"])
copie.title = "Copie Résumé"
print("Après duplication :", wb.sheetnames)

# Supprimer une feuille (exemple : copie inutile)
wb.remove(copie)
print("Après suppression :", wb.sheetnames)

# Sauvegarder le classeur
wb.save("Week 2.xlsx")

# Voir la liste des onglets
xls = pd.read_excel("Week 2.xlsx", sheet_name=None, engine="openpyxl")
print(xls.keys())

['Données employés', 'Résumé']
Après duplication : ['Données employés', 'Résumé', 'Copie Résumé']
Après suppression : ['Données employés', 'Résumé']
dict_keys(['Données employés', 'Résumé'])


Remarque :

Différence entre `wb.sheetnames` et `xls.keys()`

`wb.sheetnames` (avec openpyxl)  
- Liste les noms des feuilles sans les lire  
- Sert à renommer, supprimer ou organiser les onglets

In [85]:
from openpyxl import load_workbook
wb = load_workbook("Week 2.xlsx")
print(wb.sheetnames)
wb["Résumé"].title = "Synthèse"
print(wb.sheetnames)

['Données employés', 'Résumé']
['Données employés', 'Synthèse']


`xls.keys()` (avec pandas et sheet_name=None)  
- Liste les feuilles déjà chargées comme DataFrame  
- Sert à accéder aux données de chaque onglet

In [82]:
xls = pd.read_excel("Week 2.xlsx", sheet_name=None)
print(xls.keys())

dict_keys(['Données employés', 'Résumé'])


In [88]:

# Accéder aux données de l'onglet "Données amployés"
df_donnees = list(xls.values())[0]
df_donnees

,Nom,Age,Salaire_net,Charge (%),Salaire_brut
0,Alice,23,35000,0.25,NaN
1,Charlie,37,60000,0.25,NaN


#### Mise en forme du tableau Excel

In [90]:
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

# Charger le fichier Excel
wb = load_workbook("Week 2.xlsx")
ws = wb.active

# Définir les styles
gras_blanc = Font(bold=True, color="FFFFFF")
fond_noir = PatternFill("solid", fgColor="000000")
align_center = Alignment(horizontal="center")

bordure = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Appliquer sur l'en-tête (ligne 1)
for cell in ws[1]:
    cell.font = gras_blanc
    cell.fill = fond_noir
    cell.alignment = align_center
    cell.border = bordure

# Appliquer les formats et bordures aux lignes suivantes
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
    for cell in row:
        cell.border = bordure  # encadrer chaque cellule

    # Format pourcentage (colonne D)
    row[3].number_format = "0.00%"

    # Format monétaire (colonne C et E) avec séparateur de milliers
    row[2].number_format = '#,##0 €'
    row[4].number_format = '#,##0 €'

# Sauvegarder
wb.save("Week 2.xlsx")

Description des styles openpyxl

| Élément        | Utilité                                                       |
|----------------|----------------------------------------------------------------|
| `Font`         | Modifier la **police** : gras, taille, couleur, etc.          |
| `PatternFill`  | Appliquer un **fond coloré** à une cellule                    |
| `Alignment`    | Contrôler l’**alignement** horizontal ou vertical du contenu  |
| `Border`       | Créer une **bordure complète** autour d'une cellule           |
| `Side`         | Définir le **style d’un côté de bordure** (fin, épais, etc.)  |